**Here NAive Baayes Classifier is applied on tweets.csv dataset to classify the tweets into positive and negative. Towards the end of this notebook, testing is done using manually provided tweets**

In [0]:
#Importing Required Libraries
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
nltk.download('stopwords')
nltk.download('punkt')
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
twitter=pd.read_csv('Tweets.csv')

In [0]:
print(twitter.columns)
print(len(twitter.columns))
twitter.shape[0]
twitter.shape[1]

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')
15


15

In [0]:
print(twitter.shape[0])
print(twitter.shape[1])

14640
15


In [0]:
# This is how the Naive Bayes classifier expects the input
{'Hello': True,  'World': True}


{'Hello': True, 'World': True}

In [0]:

#removing airline names from text


airline=twitter['airline']
print(airline[0])
sree='@'+ airline[0].replace(" ","")
print(sree)
stop_airline=[]
#print(airline[100:])
for name in airline:
  name=name.replace(" ","")
  name='@'+name
  stop_airline.append(name)
  
print(len(stop_airline))



Virgin America
@VirginAmerica
14640


In [0]:
print(stop_airline[:5])

['@VirginAmerica', '@VirginAmerica', '@VirginAmerica', '@VirginAmerica', '@VirginAmerica']


In [0]:
#removing names from text

stop_name=[]
stop_name=stop_name+list(twitter['name'])
for i in range(len(stop_name)):
  stop_name[i]='@'+ stop_name[i]
print(len(stop_name))
print(stop_name[:5])

14640
['@cairdin', '@jnardino', '@yvonnalynn', '@jnardino', '@jnardino']


In [0]:
stop=[]
standard_words=list(stopwords.words("english"))
stop=standard_words+stop_name +stop_airline

In [0]:
assert len(stop)==2*14640+179

In [0]:
#Defining a function to create word features (stopwords removal etc.) and processing the input text
def create_word_features(words):
    useful_words = [word for word in words if word not in stop]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict

In [0]:
#Testing the function
create_word_features(["the", "quick", "brown", "quick", "a", "fox" ,"@VirginAmerica","@KCBobolz"])

{'brown': True, 'fox': True, 'quick': True}

In [0]:
neg_reviews = []
neg=twitter[twitter.airline_sentiment=="negative"]
print(neg[:5])
text=neg['text']
words=[]
tweet_words=[]
for word in text:
  words.append(re.sub("[^\w]", " ",  word).split())
for i in words:
  tweet_words=[]
  for j in i:
    tweet_words.append(j)
  neg_reviews.append((create_word_features(tweet_words), "negative"))


              tweet_id  ...               user_timezone
3   570301031407624196  ...  Pacific Time (US & Canada)
4   570300817074462722  ...  Pacific Time (US & Canada)
5   570300767074181121  ...  Pacific Time (US & Canada)
15  570282469121007616  ...  Pacific Time (US & Canada)
17  570276917301137409  ...  Eastern Time (US & Canada)

[5 rows x 15 columns]


In [0]:
print(neg_reviews[0])    
print(len(neg_reviews))

({'VirginAmerica': True, 'really': True, 'aggressive': True, 'blast': True, 'obnoxious': True, 'entertainment': True, 'guests': True, 'faces': True, 'amp': True, 'little': True, 'recourse': True}, 'negative')
9178


In [0]:
pos_reviews = []
pos=twitter[twitter.airline_sentiment=="positive"]
print(pos[:5])
text=pos['text']
words=[]
tweet_words=[]
for word in text:
  words.append(re.sub("[^\w]", " ",  word).split())
for i in words:
  tweet_words=[]
  for j in i:
    tweet_words.append(j)
  pos_reviews.append((create_word_features(tweet_words), "positive"))
  

              tweet_id  ...               user_timezone
1   570301130888122368  ...  Pacific Time (US & Canada)
6   570300616901320704  ...  Pacific Time (US & Canada)
8   570299953286942721  ...  Pacific Time (US & Canada)
9   570295459631263746  ...  Eastern Time (US & Canada)
11  570289724453216256  ...            America/New_York

[5 rows x 15 columns]


In [0]:
print(pos_reviews[0])    
print(len(pos_reviews))

({'VirginAmerica': True, 'plus': True, 'added': True, 'commercials': True, 'experience': True, 'tacky': True}, 'positive')
2363


In [0]:
train_set = neg_reviews[:6000] + pos_reviews[:1800]
test_set =  neg_reviews[6000:] + pos_reviews[1800:]
print(len(train_set),  len(test_set))

7800 3741


In [0]:
classifier = NaiveBayesClassifier.train(train_set)

In [0]:
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(accuracy * 100)

90.32344292969793


**Testing using manually created tweets**

In [0]:
tweet_1 = '''
 
This airline is a bad airline. I had a horrible experience.
The staff assaulted me
'''
print(tweet_1)


 
This airline is a bad airline. I had a horrible experience.
The staff assaulted me



In [0]:
words = word_tokenize(tweet_1)
words = create_word_features(words)
classifier.classify(words)

'negative'

In [0]:
tweet_2 = "This airline was too good. I had such a nice experience flying this airline"
print(tweet_2)

This airline was too good. I had such a nice experience flying this airline


In [0]:
words = word_tokenize(tweet_2)
words = create_word_features(words)
classifier.classify(words)

'positive'